# Apify Dataset | Apify Dataset> [Apify Dataset](https://docs.apify.com/platform/storage/dataset)は、製品リストやGoogleの検索結果ページ（SERP）などの構造化されたウェブスクレイピング結果を保存し、それらをJSON、CSV、Excelなど様々な形式でエクスポートするために設計された、シーケンシャルアクセスを可能とするスケーラブルな追記専用ストレージです。データセットは主に、様々なウェブスクレイピング、クローリング、データ抽出のユースケースに適用されるサーバーレスクラウドプログラムである[Apify Actors](https://apify.com/store)の結果を保存するために使用されます。>> > [Apify Dataset](https://docs.apify.com/platform/storage/dataset) is a scalable append-only storage with sequential access built for storing structured web scraping results, such as a list of products or Google SERPs, and then export them to various formats like JSON, CSV, or Excel. Datasets are mainly used to save results of [Apify Actors](https://apify.com/store)—serverless cloud programs for various web scraping, crawling, and data extraction use cases.このノートブックは、ApifyのデータセットをLangChainに読み込む方法を示しています。> This notebook shows how to load Apify datasets to LangChain.## Prerequisites | 前提条件Apifyプラットフォーム上に既存のデータセットを持っている必要があります。もし持っていない場合は、まず[このノートブック](/docs/integrations/tools/apify)をチェックして、ドキュメンテーション、ナレッジベース、ヘルプセンター、またはブログからコンテンツを抽出する方法について学んでください。> You need to have an existing dataset on the Apify platform. If you don't have one, please first check out [this notebook](/docs/integrations/tools/apify) on how to use Apify to extract content from documentation, knowledge bases, help centers, or blogs.

In [ ]:
#!pip install apify-client

まず、`ApifyDatasetLoader`をソースコードにインポートしてください：> First, import `ApifyDatasetLoader` into your source code:

In [1]:
from langchain.document_loaders import ApifyDatasetLoader
from langchain.document_loaders.base import Document

その後、ApifyデータセットレコードのフィールドをLangChainの`Document`フォーマットにマッピングする関数を提供してください。> Then provide a function that maps Apify dataset record fields to LangChain `Document` format.例えば、あなたのデータセットのアイテムがこのように構造化されているとします：> For example, if your dataset items are structured like this:```json{    "url": "https://apify.com",    "text": "Apify is the best web scraping and automation platform."}```以下のコードにおけるマッピング関数は、それらをLangChainの`Document`形式に変換し、その後任意のLLMモデル（例えば質問応答用など）で使用できるようにします。> The mapping function in the code below will convert them to LangChain `Document` format, so that you can use them further with any LLM model (e.g. for question answering).

In [3]:
loader = ApifyDatasetLoader(
    dataset_id="your-dataset-id",
    dataset_mapping_function=lambda dataset_item: Document(
        page_content=dataset_item["text"], metadata={"source": dataset_item["url"]}
    ),
)

In [ ]:
data = loader.load()

## An example with question answering | 質問応答の例この例では、データセットからのデータを使用して質問に答えます。> In this example, we use data from a dataset to answer a question.

In [4]:
from langchain.docstore.document import Document
from langchain.document_loaders import ApifyDatasetLoader
from langchain.indexes import VectorstoreIndexCreator

In [5]:
loader = ApifyDatasetLoader(
    dataset_id="your-dataset-id",
    dataset_mapping_function=lambda item: Document(
        page_content=item["text"] or "", metadata={"source": item["url"]}
    ),
)

In [ ]:
index = VectorstoreIndexCreator().from_loaders([loader])

In [7]:
query = "What is Apify?"
result = index.query_with_sources(query)

In [8]:
print(result["answer"])
print(result["sources"])

 Apify is a platform for developing, running, and sharing serverless cloud programs. It enables users to create web scraping and automation tools and publish them on the Apify platform.

https://docs.apify.com/platform/actors, https://docs.apify.com/platform/actors/running/actors-in-store, https://docs.apify.com/platform/security, https://docs.apify.com/platform/actors/examples
